In [9]:
import subprocess
import pandas as pd
import json
import shutil
import mlflow

def generate_dockerfile(model_name, model_version):
    model_uri = f"models:/{model_name}/{model_version}"
    subprocess.run(["mlflow", "models", "generate-dockerfile", "-m", model_uri])

def generate_input_json(filename, n_samples=5):
    
    df = pd.read_csv(filename)
    request = df.head(n_samples).to_dict('records')

    data = {"dataframe_records": request}

    with open('input.json', 'w') as f:
        json.dump(data, f)
        

def generate_predictions(model_name, model_version, X_inference):
    model = mlflow.sklearn.load_model(model_uri=f"models:/{model_name}/{model_version}")
    predictions = model.predict_proba(X_inference).tolist()  # Converting numpy array to list
    print(f"{model.predict_proba(X_inference, )[0]}")
    print(f"{len(model.predict_proba(X_inference)[0])}")
    print(f"{model.predict(X_inference)}")
    output = json.dumps({"predictions": predictions})  # Converting list to JSON
    with open('predictions.txt', 'w') as f:
        f.write(f"{output}\n")

            
def generate_instructions(model_name, model_version):
    instructions = (
        f"# Para crear la imagen Docker ejecutar el siguiente comando:\n"
        f"docker build -t {model_name.lower()}_{model_version} .\n\n"
        f"# Para iniciar la imagen Docker como API del modelo, ejecutar:\n"
        f"docker run -it -p 5002:8080 {model_name.lower()}_{model_version}\n\n"
        f"# Para probar el API del modelo creado, ejecutar:\n"
        "curl -X POST -H \"Content-Type:application/json\" -d @input.json http://localhost:5002/invocations > api_predictions.txt && printf '\\n' >> api_predictions.txt\n\n"
        "# Las predicciones en api_predictions.txt deben coincidir con el archivo predictions.txt\n"
        "# Opcionalmente puedes ejecutar el siguiente comando para ver si hay diferencias.\n"
        "diff -w api_predictions.txt predictions.txt"
    )
    with open("instructions.txt", "w") as f:
        f.write(instructions)


def create_zip():
    shutil.copy('input.json', 'mlflow-dockerfile')
    shutil.copy('predictions.txt', 'mlflow-dockerfile')
    shutil.copy('instructions.txt', 'mlflow-dockerfile')
    shutil.make_archive('model', 'zip', 'mlflow-dockerfile')

model_name = "extratree"  
model_version = "20"
filename = "X_inference.csv"

n_samples = 25
generate_dockerfile(model_name, model_version)
generate_input_json(filename, n_samples=n_samples)
X_inference = pd.read_csv(filename).head(n_samples)
generate_predictions(model_name, model_version, X_inference)
generate_instructions(model_name, model_version)
create_zip()


[0.   0.16 0.   0.   0.   0.   0.12 0.   0.   0.   0.   0.   0.02 0.12
 0.   0.06 0.   0.   0.   0.06 0.08 0.   0.   0.   0.   0.32 0.06 0.
 0.   0.  ]
30
[32 11 12  9  1  1 36 27 13 29  9  8  1 31  7 24 13 34 14 24  1  1 34 34
  8]
